<a href="https://colab.research.google.com/github/Zaedul-Islam/Machine_Learning_Projects/blob/master/Car%20Acceptability%20Evaluation/Car%20Acceptability%20Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Acceptability Evaluation**

## **Importing the libraries**

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Seaborn is a python library for visualizing data. It is built on top of matplotlib and closely integrated with pandas data structure
import seaborn as sns

# Some setting for this notebook to actually show the graphs inline
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)  # Suppress scientific float notation

## **Reading 'cereal' dataset**

In [5]:
filepath = '/content/drive/My Drive/Colab Notebooks/Machine Learning Projects/Car Acceptability Evaluation/Dataset/car.csv'
dataset = pd.read_csv(filepath)

dataset.head(5)

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


**Findings**:
1. No missing values in the dataset and all are ***Categorical*** attributes.